# Test the added function of the class DistanceMetrics in validation.py

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import os
from pathlib import Path
ROOT_DIR = Path(os.path.dirname(os.path.abspath(".gitignore"))).parent
os.chdir(ROOT_DIR)

In [8]:
# Load neccessary modules
import validation
import pandas as pd

In [12]:
df = pd.read_csv(str(ROOT_DIR) + '/results/20200901_202859/distance-metrics-east.csv')
df.head()

,distance,model_mean,gravity_seed_mean,gravity_mean,sampers_mean,model_variance,gravity_seed_variance,gravity_variance,sampers_variance,model_sum,gravity_seed_sum,gravity_sum,sampers_sum
0,"(-0.001, 3.211]",7.579992e-06,5.927266e-07,2.143508e-06,7.370865e-06,4.455201e-09,2.259008e-16,1.147418e-10,1.943457e-08,0.699603,0.054706,0.197837,0.680301
1,"(3.211, 5.192]",9.530627e-07,5.532856e-07,1.170618e-06,7.699187e-07,4.115526e-11,9.021649e-17,2.101613e-11,2.181541e-11,0.087962,0.051065,0.108041,0.071059
2,"(5.192, 6.939]",5.352549e-07,5.235326e-07,7.755889e-07,4.483360e-07,1.414810e-11,6.232140e-17,6.314211e-12,1.681111e-11,0.049401,0.048319,0.071582,0.041379
3,"(6.939, 8.647]",3.324327e-07,4.971497e-07,5.562334e-07,2.938864e-07,8.585287e-12,5.433544e-17,3.225533e-12,4.295125e-12,0.030682,0.045884,0.051337,0.027124
4,"(8.647, 10.313]",2.259872e-07,4.725876e-07,4.296217e-07,2.514278e-07,7.011046e-12,4.638674e-17,1.797816e-12,1.158260e-11,0.020858,0.043618,0.039652,0.023206


In [19]:
# Sampers vs the proposed model
ds = validation.DistanceMetrics()
print(ds.kullback_leibler(distances=df, titles =['sampers', 'model']))

0.00861413722338568


In [22]:
# Sampers vs gravity model
ds = validation.DistanceMetrics()
print(ds.kullback_leibler(distances=df, titles =['sampers', 'gravity']))

0.26008258045198923
